In [1]:
import numpy as np
import pandas as pd

In [2]:
# 数据聚合与分组运算
# 先来看看这个非常简单的表格型数据集
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.357141,0.347548
1,a,two,-0.832734,0.851675
2,b,one,1.912889,-1.222742
3,b,two,-0.337824,0.365060
4,a,one,0.836447,0.264417


In [3]:
# 假设你想要按key1进行分组，并计算data1列的平均值。实现该功能的方式有很多，而我们
# 这里要用的是：访问data1，并根据key1调用groupby
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# 变量grouped是一个GroupBy对象。它实际上还没有进行任何运算，只是含有一些有关
# 分组键df['key1']的中间数据而已，换句话说，该对象已经有了接下来对各组执行运算
# 的一切信息，我们可以调用GroupBy的mean方法计算分组平均值
grouped.mean()

key1
a   -0.451143
b    0.787533
Name: data1, dtype: float64

In [5]:
# 这里最重要的是，数据（Series）根据分组键进行了聚合，产生了一个新的Series，其索引
# key1列中的唯一值。之所以结果中索引的名称为key1，是因为原始DataFrame的列就叫这个

In [6]:
# 如果我们一次传入多个数组的列表，就会得到不同的结果
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.260347
      two    -0.832734
b     one     1.912889
      two    -0.337824
Name: data1, dtype: float64

In [7]:
# 这里我们通过两个键对数据进行了分组，得到的Series具有一个层次化索引
means.unstack()

key2,one,two
key1,,
a,-0.260347,-0.832734
b,1.912889,-0.337824


In [8]:
# 在这个例子中，分组键均为Series。实际上，分组键可以是任何长度适当的数组
# states years分别对应data1中的值
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.832734
            2006    1.912889
Ohio        2005   -0.847482
            2006    0.836447
Name: data1, dtype: float64

In [9]:
# 通常，分组信息就位于相同的要处理DataFrame中。这里，你可以将列名（可以是字符串、
# 数字或其他Python对象）用作分组键
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.451143,0.487880
b,0.787533,-0.428841


In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.260347  0.305982
     two  -0.832734  0.851675
b    one   1.912889 -1.222742
     two  -0.337824  0.365060

In [11]:
# 第一个例子在执行df.groupby('key1').mean()时，结果中没有key2列，这是因为df['key2']
# 不是数值数据（俗称“麻烦列”）所以从结果中排除了。默认情况下，所有数值列都会被
# 聚合，虽然有时可能会被过滤为一个子集，稍后就会碰到

In [12]:
# 无论你准备拿groupby做什么，都有可能会用到Groupby的size方法，它可以返回一个
# 含有分组大小的Series
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [13]:
# 任何分组关键词中的缺失值都会被从结果中除去

In [14]:
#groupby对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.357141  0.347548
1    a  two -0.832734  0.851675
4    a  one  0.836447  0.264417
b
  key1 key2     data1     data2
2    b  one  1.912889 -1.222742
3    b  two -0.337824  0.365060


In [15]:
# 对于多重键的情况，元组的第一个元素将会是由键值组成的元组
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.357141  0.347548
4    a  one  0.836447  0.264417
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.832734  0.851675
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.912889 -1.222742
('b', 'two')
  key1 key2     data1    data2
3    b  two -0.337824  0.36506


In [16]:
# 你可以对这些数据片段做任何操作。比如将这些数据片段做成一个字典
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -1.357141  0.347548
 1    a  two -0.832734  0.851675
 4    a  one  0.836447  0.264417, 'b':   key1 key2     data1     data2
 2    b  one  1.912889 -1.222742
 3    b  two -0.337824  0.365060}

In [17]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-1.357141,0.347548
1,a,two,-0.832734,0.851675
4,a,one,0.836447,0.264417


In [18]:
# groupby默认是在axis=0上进行分组的，通过设置也可以在其他任何轴上进行分组。
# 如 可以根据dtype对列进行分组
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [19]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.357141  0.347548
1 -0.832734  0.851675
2  1.912889 -1.222742
3 -0.337824  0.365060
4  0.836447  0.264417
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [20]:
# 对于DataFrame产生的Groupby对象，如果用一个（单个字符串）或一组（字符串数组）
# 列名对其进行索引，就能实现选取部分列进行聚合的目的
df.groupby('key1')['data1']

In [21]:
df.groupby('key1')[['data2']]

In [22]:
# 是以下代码的语法糖
df['data1'].groupby(df['key1'])

In [23]:
df[['data2']].groupby(df['key1'])

In [24]:
# 尤其对于大数据集，很可能只需要对部分列进行聚合。例如，在前面那个数据集中，如果
# 只需计算data2列的平均值并以DataFrame形式得到结果，可以这样写
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.305982
     two   0.851675
b    one  -1.222742
     two   0.365060

In [25]:
# 这种索引操作所返回的对象是一个已分组的DataFrame（传入列表或数组）或已分组的
# Series（传入标量形式的单个列名）
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [26]:
s_grouped.mean()

key1  key2
a     one     0.305982
      two     0.851675
b     one    -1.222742
      two     0.365060
Name: data2, dtype: float64

In [27]:
s_grouped.mean().unstack()

key2,one,two
key1,,
a,0.305982,0.851675
b,-1.222742,0.365060


In [28]:
# 通过字典或Series进行分组
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,0.307458,-2.006146,-0.279938,0.532983,1.722575
Steve,-0.225212,-0.112758,2.719801,2.050350,-0.280634
Wes,-0.014054,NaN,NaN,0.880176,0.432590
Jim,1.010629,1.517396,0.245413,0.707291,0.439866
Travis,0.856504,0.005379,1.362357,0.223980,0.501191


In [29]:
# 现在，假设已知列的分组关系，并希望根据分组计算列的和
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}
# 将这个字典传给groupby来构造数组，但我们可以直接传递字典（f说明存在未使用的分组键也是可以的）
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.253045,0.023887
Steve,4.770151,-0.618604
Wes,0.880176,0.418536
Jim,0.952704,2.967891
Travis,1.586337,1.363074


In [30]:
# Series也有同样的功能，它可以被看做一个固定大小的映射
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [31]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [32]:
# 通过函数进行分组
people.groupby(len).sum()

,a,b,c,d,e
3,1.304033,-0.488750,-0.034525,2.12045,2.595031
5,-0.225212,-0.112758,2.719801,2.05035,-0.280634
6,0.856504,0.005379,1.362357,0.22398,0.501191


In [33]:
# 将函数跟数组、列表、字典、Series混合使用也不是问题
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.014054 -2.006146 -0.279938  0.532983  0.432590
  two  1.010629  1.517396  0.245413  0.707291  0.439866
5 one -0.225212 -0.112758  2.719801  2.050350 -0.280634
6 two  0.856504  0.005379  1.362357  0.223980  0.501191

In [34]:
people

,a,b,c,d,e
Joe,0.307458,-2.006146,-0.279938,0.532983,1.722575
Steve,-0.225212,-0.112758,2.719801,2.050350,-0.280634
Wes,-0.014054,NaN,NaN,0.880176,0.432590
Jim,1.010629,1.517396,0.245413,0.707291,0.439866
Travis,0.856504,0.005379,1.362357,0.223980,0.501191


In [35]:
# 层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.412434 -0.467918  0.584132 -0.535070  1.619155
1      2.244178 -1.441859 -0.321660  0.130123  0.575539
2      1.096993 -0.026782  1.151694 -0.604167  0.388770
3      0.467878  0.928275 -0.617805  0.639632 -1.152377

In [36]:
# 要根据级别分组，使用level关键字传递级别序号或名字
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [37]:
# 数据聚合指的是任何能够从数组产生标量值的数据转换过程。之前的例子已经用过一些。
# 比如mean、count、min、sum。
# 你可以使用自己发明的聚合运算，还可以调用分组对象上已经定义好的任何方法，例如，
# quantile可以计算Series或DataFrame列的样本分位数
df

,key1,key2,data1,data2
0,a,one,-1.357141,0.347548
1,a,two,-0.832734,0.851675
2,b,one,1.912889,-1.222742
3,b,two,-0.337824,0.365060
4,a,one,0.836447,0.264417


In [41]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.5)
# b中二分位数是-0.3到1.9的‘中位数’ a以此类推

key1
a   -0.832734
b    0.787533
Name: data1, dtype: float64

In [42]:
# 如果要使用你自己的聚合函数，只需将其传入aggregate或agg方法
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.193588,0.587258
b,2.250713,1.587802


In [43]:
# 你可能注意到，有些方法（如describe）也是可以用在这里的，即使严格来讲，他们非聚合运算
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.451143  1.145498 -1.357141 -1.094937 -0.832734  0.001856   
b      2.0  0.787533  1.591494 -0.337824  0.224855  0.787533  1.350211   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.836447   3.0  0.487880  0.317786  0.264417  0.305982  0.347548   
b     1.912889   2.0 -0.428841  1.122746 -1.222742 -0.825792 -0.428841   

                          
           75%       max  
key1                      
a     0.599611  0.851675  
b    -0.031891  0.365060

In [46]:
# 面向列的多函数应用  用一个小费百分比的列tip_pct
tips = pd.read_csv('F:/sub+pro+ject/AI/Data Analysis/examples/examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [47]:
# 你已经看到，对Series或DataFrame列的聚合运算其实就是使用aggregate或调用诸如mean、
# std之类的方法。然而，你可能希望对不同的列使用不同的聚合函数，或一次应用多个函数
# 先根据day和smoker对tips进行分组
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
# 内置的函数名可以用字符串传入
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [48]:
grouped['tip_pct'].mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [49]:
# 如果传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名
grouped_pct.agg(['mean', 'std', 'peak_to_peak'])

AttributeError: 'SeriesGroupBy' object has no attribute 'peak_to_peak'

In [50]:
# 如果传入一组函数或函数名，得到的DataFrame的列就会以相应的函数命名
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240